## Notebook contains loading of dataset, it's transformations and data extraction for further work

### Step no. 0: loading csv and other data

In [1]:
# !pipenv shell
# !pipenv --where
# !python --version

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

import utils

In [4]:
root_dir = "../../"
data_dir = os.path.join(root_dir, "datasets/")
processed_data_dir = os.path.join(data_dir, "processed_data/")
new_hydro_dir = os.path.join(data_dir, "hydro_2018-2020")

water_levels_path = os.path.join(new_hydro_dir, "new_data_all.csv")
water_levels = pd.read_csv(water_levels_path, sep=";")

posts_path = os.path.join(processed_data_dir, "asunp.pkl")
posts = utils.load_pickle(posts_path)

#### показываем данные

In [5]:
water_levels.head()
posts.head()

,time,max_level,identifier
0,2020-10-01 00:00:00,232.0,5116
1,1986-06-20 00:00:00,278.0,5292
2,1986-06-22 00:00:00,67.0,6022
3,1986-06-23 00:00:00,323.0,5216
4,1986-06-22 00:00:00,112.0,5094


,foId,pgid,pagr,paer,codBasin,station_id,kod1,kod2,kod3,agro,...,cgms,pn,stStatus,rv,pmet,codeSubject,ter,parentNpSn,kto,geometry
0,8.0,2.0,0.0,0.0,177,<NA>,639,445,000,NaN,...,999,100.0,open,0,0.0,14,98,5043.0,53,POINT (141.36667 63.33333)
1,8.0,4.0,0.0,0.0,177,<NA>,639,151,000,NaN,...,999,20.0,open,0,0.0,14,98,5049.0,53,POINT (144.30000 64.44667)
2,8.0,2.0,0.0,0.0,200,2224,619,000,000,NaN,...,999,28.0,open,0,0.0,41,30,5333.0,53,POINT (160.83000 56.81000)
3,8.0,3.0,0.0,0.0,181,5674,605,405,000,NaN,...,999,13.0,open,0,0.0,79,99,4316.0,53,POINT (132.80000 48.72000)
4,2.0,2.0,0.0,0.0,010,71104,605,161,000,NaN,...,999,21.0,open,0,1.0,51,47,399.0,53,POINT (33.08000 68.83000)


### Step no. 1: build light dataframes to find correlations between water levels
#### Needed: time series of water levels, coordinates of posts

In [5]:
water_datetime_date = pd.to_datetime(water_levels["time"], format="%Y-%m-%d %H:%M:%S").dt.date
water_levels["time"] = water_datetime_date

water_levels.rename(columns={"time": "date", "identifier": "id"}, inplace=True)

water_levels.sort_values(by=["id", "date"], inplace=True)
water_levels.set_index(["id", "date"], inplace=True)

water_levels.head()

max_level
id   date                 
5001 1984-01-01      258.0
     1984-01-02      255.0
     1984-01-03      252.0
     1984-01-04      248.0
     1984-01-05      244.0

In [6]:
all_hydro_posts_ids = posts["gidro"].astype(float) # elem is None if post doesn't belong to hydro type
posts["gidro"] = all_hydro_posts_ids

needed_hydro_posts_ids = water_levels.index.get_level_values("id").unique()

keep_hydro_posts_mask = all_hydro_posts_ids.isin(needed_hydro_posts_ids)
hydro_posts = posts[keep_hydro_posts_mask].reset_index(drop=True)
hydro_posts = hydro_posts[["gidro", "lat", "lon"]]

# drop multiple occurencies of single gidro id, because it can be assigned to several united posts
# Because we need only coords of post
hydro_posts = hydro_posts.drop_duplicates(subset="gidro")

hydro_posts.rename(columns={"gidro": "id", "lat": "latitude", "lon": "longitude"}, inplace=True)
hydro_posts.set_index("id", inplace=True)

hydro_posts.head()

,latitude,longitude
id,,
5674.0,48.72,132.80
5216.0,44.06,132.01
6256.0,52.17,126.58
5132.0,44.23,134.27
5151.0,43.58,133.04


In [33]:
# dumping ready data
working_data_dir = os.path.join(root_dir, "working_data/")
os.makedirs(working_data_dir, exist_ok=True)

hydro_posts_path = os.path.join(working_data_dir, "hydro_posts_coords.csv")
water_level_path = os.path.join(working_data_dir, "water_levels.csv")

hydro_posts.to_csv(hydro_posts_path)
water_levels.to_csv(water_level_path)

### Step no. 2: table to convert between meteo_id and serialNum 

In [45]:
meteo_convert_id_number = posts[["meteo_id", "station_id"]]

has_nan_mask = meteo_convert_id_number.isna().sum(axis=1) == 0
meteo_convert_id_number = meteo_convert_id_number[has_nan_mask]

meteo_convert_id_number = meteo_convert_id_number[meteo_convert_id_number["meteo_id"].notna()].set_index("meteo_id", drop=True)
meteo_convert_id_number = meteo_convert_id_number.loc[~meteo_convert_id_number.index.duplicated(keep='first')]

meteo_convert_id_number.to_csv(working_data_dir + "meteo_id_to_number.csv")

In [47]:
meteo_convert_id_number

,station_id
meteo_id,
6967071,25057
5239090060,9207
5389140,29865
545705016,77217
5679600,29485
...,...
5235054000,57604
6001032390,72169
4622042040,59304


### building own s2m from scratch because it contains nans

In [54]:
s2m = pd.read_pickle(processed_data_dir + "s2m.pkl")
s2m.head(2)

,meteo_id,dist
station_id,,
5674,4943321,81.846330
5216,4433241,21.467114


In [56]:
posts_path = os.path.join(processed_data_dir, "asunp.pkl")
posts = utils.load_pickle(posts_path)
posts.head(2)

,foId,pgid,pagr,paer,codBasin,station_id,kod1,kod2,kod3,agro,...,cgms,pn,stStatus,rv,pmet,codeSubject,ter,parentNpSn,kto,geometry
0,8.0,2.0,0.0,0.0,177,<NA>,639,445,000,NaN,...,999,100.0,open,0,0.0,14,98,5043.0,53,POINT (141.36667 63.33333)
1,8.0,4.0,0.0,0.0,177,<NA>,639,151,000,NaN,...,999,20.0,open,0,0.0,14,98,5049.0,53,POINT (144.30000 64.44667)


In [79]:
def load_col_unique_vals(path, col):
    column = pd.read_csv(path, usecols=[col])
    return pd.unique(column.values.flatten())
    
used_meteo_nums = load_col_unique_vals(working_data_dir + "meteo_features.csv", "stationNumber")
used_hydro_stations = load_col_unique_vals(working_data_dir + "water_levels.csv", "id")

In [130]:
all_meteo_nums = posts["station_id"]
filtered_meteo_data = posts[all_meteo_nums.isin(used_meteo_nums)]

used_meteo_coords = filtered_meteo_data[["station_id", "lon", "lat"]].reset_index(drop=True)
used_meteo_coords.set_index("station_id", drop=True, inplace=True)
used_meteo_coords.head()

,lon,lat
station_id,,
30859,114.52,51.10
31594,130.08,49.42
31510,127.48,50.28
31439,140.47,52.38
31253,128.87,54.72


In [131]:
all_hydro_ids = posts["gidro"].astype(float)
posts["gidro"] = all_hydro_ids

filtered_hydro_data = posts[all_hydro_ids.isin(used_hydro_stations)]

used_hydro_coords = filtered_hydro_data[["gidro", "lon", "lat"]].reset_index(drop=True)

used_hydro_coords = used_hydro_coords.loc[~used_hydro_coords["gidro"].duplicated(keep='first')]

used_hydro_coords.set_index("gidro", drop=True, inplace=True)
used_hydro_coords.head()

,lon,lat
gidro,,
5674.0,132.80,48.72
5216.0,132.01,44.06
6256.0,126.58,52.17
5132.0,134.27,44.23
5151.0,133.04,43.58


In [136]:
def distances(row, df):
    df_lons, df_lats = df["lon"], df["lat"]
    row_lon, row_lat = row["lon"], row["lat"]
    dist = (row_lon - df_lons) ** 2 + (row_lat - df_lats) ** 2
    dist = dist ** 0.5
    return dist

hydro_meteo_dists = used_hydro_coords.apply(lambda x: distances(x, used_meteo_coords), axis=1)
my_s2m = hydro_meteo_dists.idxmin(axis=1)
my_s2m

gidro
5674.0    31725
5216.0    31915
6256.0    31371
5132.0    31931
5151.0    31931
          ...  
6363.0    31329
5155.0    31931
5765.0    31931
5296.0    31873
5761.0    31931
Length: 198, dtype: int64

In [137]:
my_s2m.index.name = "station_id"
my_s2m.name = "meteo_id"

In [138]:
my_s2m.to_csv("./handmade_s2m.csv")

In [139]:
hydro_meteo_dists.iloc[3]

station_id
30859    20.910748
31594     6.670247
31510     9.094317
31439    10.240239
31253    11.798309
30965    18.797082
31873     1.533949
30695    13.889366
31707     4.812831
31733     4.864175
31416     8.488168
31586     8.291086
31845     2.283528
30949    22.930451
31788     2.920616
31443     9.190348
30954    22.281851
31895     1.463728
30673    17.379356
30879    16.275460
31562     7.748819
31388     9.211781
31915     2.911668
31931     1.245351
30692    14.196581
31725     4.392095
31677     6.220289
31478     8.045228
30777    18.400133
31538     6.219743
31735     4.395013
31532     6.940461
31371    11.899676
31329     8.950871
Name: 5132.0, dtype: float64

In [135]:
hydro_meteo_dists.idxmin(axis=1)

gidro
5674.0    31725
5216.0    31915
6256.0    31371
5132.0    31931
5151.0    31931
          ...  
6363.0    31329
5155.0    31931
5765.0    31931
5296.0    31873
5761.0    31931
Length: 198, dtype: int64